In [1]:
# IMPORTS
from dataset_manager import Dataset_Manager
import io
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GroupKFold
from sklearn.metrics import confusion_matrix
from sklearn.neural_network import MLPClassifier
import subprocess
import sys

In [2]:
# LOAD DATASETS
dataset_manager = Dataset_Manager()
RECOLA_DATASET = dataset_manager.recola_dataset
AGAIN_DATASET = dataset_manager.again_dataset

In [3]:
# GET INVARIANT FEATURES
def get_invariant_features(input_file: str, output_file: str):
    RSCRIPT = "C:/Program Files/R/R-4.4.3/bin/x64/Rscript.exe"
    INVARIANT_FEATURES_FILE = "invariant_features.R" 
    run_params = [RSCRIPT, INVARIANT_FEATURES_FILE, "inv_pred_recola", input_file, output_file]

    # PRINTING RSCRIPT PARAMETERS DETAILS
    print("-------- R Script Parameters --------")
    print(f" Rscript Path:\t\t\'{run_params[0]}\'")
    print(f" R File Path:\t\t\'{run_params[1]}\'")
    print(f" Function Name:\t\t\'{run_params[2]}\'")
    print(f" Passed Parameters:\t\'{run_params[3:]}\'")

    try:
        subprocess.run(run_params, check=True, capture_output=True, text=True)
    except subprocess.CalledProcessError as e: 
        print("Error running R script:", e)

    invariant_features_RECOLA = pd.read_csv(output_file)
    return list(invariant_features_RECOLA["invariant_feature_names"])

In [4]:
# LOGISTIC REGRESSION
def logistic_regression(groups: list, features: pd.DataFrame, target: pd.Series, folds: int):
    conf_matrix_list = []

    group_k_fold = GroupKFold(n_splits=folds)

    for train_index, test_index in group_k_fold.split(features, target, groups):
        # Training split
        feature_train = features.iloc[train_index]
        target_train = target.iloc[train_index]
        group_train = np.array(groups)[train_index]

        # Drop NaNs in training data
        train_mask = ~feature_train.isna().any(axis=1)
        feature_train = feature_train[train_mask]
        target_train = target_train[train_mask]
        group_train = group_train[train_mask]

        # Testing split (don't drop NaNs unless you want to)
        feature_test = features.iloc[test_index]
        target_test = target.iloc[test_index]

        # Drop NaNs in test set too, to avoid prediction errors
        test_mask = ~feature_test.isna().any(axis=1)
        feature_test = feature_test[test_mask]
        target_test = target_test[test_mask]

        if feature_train.empty or feature_test.empty:
            continue

        model = LogisticRegression(solver='liblinear', max_iter=100000)
        model.fit(X=feature_train, y=target_train)

        test_prediction = model.predict(feature_test)
        conf_matrix = confusion_matrix(target_test, test_prediction)
        conf_matrix_list.append(conf_matrix)

    # Averaging confusion matrices
    mean_confusion_matrix = np.mean(conf_matrix_list, axis=0)
    true_neg, false_pos, _, true_pos = mean_confusion_matrix.ravel()

    mean_accuracy = (true_pos + true_neg) / np.sum(mean_confusion_matrix)
    mean_precision = true_pos / (true_pos + false_pos) if (true_pos + false_pos) else 0

    return mean_accuracy, mean_precision

def run_logistic_regression(participant_groups, features, invariant_features, target_label, folds):
    print("\n-------- Logistic Regression Results --------\n")
    all_accuracy, all_precision = logistic_regression(groups=participant_groups, features=features, target=target_label, folds=folds)
    inv_accuracy, inv_precision = logistic_regression(groups=participant_groups, features=invariant_features, target=target_label, folds=folds)

    print(f" COMPARING BETWEEN ALL FEATURES AND ONLY INVARIANT FEATURES")
    print(f" ACCURACY:\t{all_accuracy} -> {inv_accuracy}")
    print(f" PRECISION:\t{all_precision} -> {inv_precision}")

In [5]:
# NEURAL NETWORKS
def neural_networks(groups: list, features: pd.DataFrame, target: pd.Series, folds: int):
    conf_matrix_list = []
    
    group_k_fold = GroupKFold(n_splits=folds)
    for train_index, test_index in group_k_fold.split(features, target, groups):
        # Training split
        feature_train = features.iloc[train_index]
        target_train = target.iloc[train_index]
        group_train = np.array(groups)[train_index]

        # Drop NaNs in training data
        train_mask = ~feature_train.isna().any(axis=1)
        feature_train = feature_train[train_mask]
        target_train = target_train[train_mask]
        group_train = group_train[train_mask]

        # Testing split (don't drop NaNs unless you want to)
        feature_test = features.iloc[test_index]
        target_test = target.iloc[test_index]

        # Drop NaNs in test set too, to avoid prediction errors
        test_mask = ~feature_test.isna().any(axis=1)
        feature_test = feature_test[test_mask]
        target_test = target_test[test_mask]

        if feature_train.empty or feature_test.empty:
            continue

        model = MLPClassifier(hidden_layer_sizes=(32,), max_iter=10000, random_state=42)
        model.fit(X=feature_train, y=target_train)

        test_prediction = model.predict(feature_test)
        conf_matrix = confusion_matrix(target_test, test_prediction)
        conf_matrix_list.append(conf_matrix)
        
    mean_confusion_matrix = np.mean(conf_matrix_list, axis=0)
    true_neg, false_pos, _, true_pos = mean_confusion_matrix.ravel()

    mean_accuracy = (true_pos + true_neg) / np.sum(mean_confusion_matrix)
    mean_precision = true_pos / (true_pos + false_pos)

    return mean_accuracy, mean_precision

def run_neural_networks(participant_groups, features, invariant_features, target_label, folds):
    print("\n-------- Neural Networks Results --------\n")
    all_accuracy, all_precision = neural_networks(groups=participant_groups, features=features, target=target_label, folds=folds)
    inv_accuracy, inv_precision = neural_networks(groups=participant_groups, features=invariant_features, target=target_label, folds=folds)

    print(f" COMPARING BETWEEN ALL FEATURES AND ONLY INVARIANT FEATURES")
    print(f" ACCURACY:\t{all_accuracy} -> {inv_accuracy}")
    print(f" PRECISION:\t{all_precision} -> {inv_precision}")

In [6]:
# TEST CASE 1: SINGLE MODALITY (AUDIO), 6 PARTICIPANTS, VALENCE
output_file = "RECOLA_Test_Case_1"
buffer = io.StringIO()
sys.stdout = buffer

print("-------- TEST CASE 1: SINGLE MODALITY (AUDIO), 6 PARTICIPANTS, TARGET LABEL: VALENCE --------\n")

# STEP 1: OBTAIN DATASET 
dataset_manager.temproary_recola = dataset_manager.get_number_of_participants_frame(number_of_participants=6)
dataset_manager.temproary_recola = dataset_manager.remove_modality(modalities=["Video", "Physiology"])
dataset_manager.temproary_recola = dataset_manager.remove_class_label(label_to_keep="Valence")
dataset = dataset_manager.save_custom_dataframe(file_name=f"{output_file}.csv")

# STEP 2: OBTAIN INVARIANT FEATURES
INPUT_FILE = f"Formatted_Datasets/{output_file}.csv"
OUTPUT_FILE = f"Invariant_Features/inv_feat_{output_file}.csv"

RECOLA_invariant_list = get_invariant_features(input_file=INPUT_FILE, output_file=OUTPUT_FILE)

if len(RECOLA_invariant_list) > 0:
    print("\n-------- Invariant Features --------")
    for feature in RECOLA_invariant_list:
        print(f" {feature}")

    # STEP 3: MODEL PREP
    PARTICIPANT_GROUPS = list(dataset["Participant_Number"])
    FEATURES = dataset.filter(regex=f'^{"ComPar"}|{"audio_speech"}|{"VIDEO"}|{"Face_detection"}|{"ECG"}|{"EDA"}', axis=1) 
    INVARIANT_FEATURES = dataset.filter(RECOLA_invariant_list, axis=1)           
    if "Class_Label_Valence" in dataset.columns: TARGET_LABEL = dataset["Class_Label_Valence"]
    elif "Class_Label_Arousal" in dataset.columns: TARGET_LABEL = dataset["Class_Label_Arousal"]
    FOLDS = len(set(PARTICIPANT_GROUPS))

    # STEP 4: RUN LOGISITIC REGRESSION EXPERIMENTS
    run_logistic_regression(PARTICIPANT_GROUPS, FEATURES, INVARIANT_FEATURES, TARGET_LABEL, FOLDS)

    # STEP 5: RUN NEURAL NETWORK EXPERIMENTS
    run_neural_networks(PARTICIPANT_GROUPS, FEATURES, INVARIANT_FEATURES, TARGET_LABEL, FOLDS)
else:
    print("No Invariant Features were found")

# STEP 6: SAVE OUTPUT
sys.stdout = sys.__stdout__
with open(f"Test_Results/{output_file}.txt", "w") as f:
    f.write(buffer.getvalue())

In [7]:
# TEST CASE 2: MALE PARTICIPANTS, TARGET LABEL: AROUSAL
output_file = "RECOLA_Test_Case_2"
buffer = io.StringIO()
sys.stdout = buffer

print("-------- TEST CASE 2: MALE PARTICIPANTS, TARGET LABEL: AROUSAL --------\n")

# STEP 1: OBTAIN DATASET 
dataset_manager.temproary_recola = dataset_manager.split_by_gender(gender="Male")
dataset_manager.temproary_recola = dataset_manager.remove_class_label(label_to_keep="Arousal")
dataset = dataset_manager.save_custom_dataframe(file_name=f"{output_file}.csv")

# STEP 2: OBTAIN INVARIANT FEATURES
INPUT_FILE = f"Formatted_Datasets/{output_file}.csv"
OUTPUT_FILE = f"Invariant_Features/inv_feat_{output_file}.csv"

RECOLA_invariant_list = get_invariant_features(input_file=INPUT_FILE, output_file=OUTPUT_FILE)

if len(RECOLA_invariant_list) > 0:
    print("\n-------- Invariant Features --------")
    for feature in RECOLA_invariant_list:
        print(f" {feature}")

    # STEP 3: MODEL PREP
    PARTICIPANT_GROUPS = list(dataset["Participant_Number"])
    FEATURES = dataset.filter(regex=f'^{"ComPar"}|{"audio_speech"}|{"VIDEO"}|{"Face_detection"}|{"ECG"}|{"EDA"}', axis=1) 
    INVARIANT_FEATURES = dataset.filter(RECOLA_invariant_list, axis=1)           
    if "Class_Label_Valence" in dataset.columns: TARGET_LABEL = dataset["Class_Label_Valence"]
    elif "Class_Label_Arousal" in dataset.columns: TARGET_LABEL = dataset["Class_Label_Arousal"]
    FOLDS = len(set(PARTICIPANT_GROUPS))

    # STEP 4: RUN LOGISITIC REGRESSION EXPERIMENTS
    run_logistic_regression(PARTICIPANT_GROUPS, FEATURES, INVARIANT_FEATURES, TARGET_LABEL, FOLDS)

    # STEP 5: RUN NEURAL NETWORK EXPERIMENTS
    run_neural_networks(PARTICIPANT_GROUPS, FEATURES, INVARIANT_FEATURES, TARGET_LABEL, FOLDS)

else:
    print("\n-------- No Invariant Features were found --------")

# STEP 6: SAVE OUTPUT
sys.stdout = sys.__stdout__
with open(f"Test_Results/{output_file}.txt", "w") as f:
    f.write(buffer.getvalue())

In [8]:
# TEST CASE 3: MALE PARTICIPANTS, TARGET LABEL: VALENCE
output_file = "RECOLA_Test_Case_3"
buffer = io.StringIO()
sys.stdout = buffer

print("-------- TEST CASE 3: MALE PARTICIPANTS, TARGET LABEL: VALENCE --------\n")

# STEP 1: OBTAIN DATASET 
dataset_manager.temproary_recola = dataset_manager.split_by_gender(gender="Male")
dataset_manager.temproary_recola = dataset_manager.remove_class_label(label_to_keep="Valence")
dataset = dataset_manager.save_custom_dataframe(file_name=f"{output_file}.csv")

# STEP 2: OBTAIN INVARIANT FEATURES
INPUT_FILE = f"Formatted_Datasets/{output_file}.csv"
OUTPUT_FILE = f"Invariant_Features/inv_feat_{output_file}.csv"

RECOLA_invariant_list = get_invariant_features(input_file=INPUT_FILE, output_file=OUTPUT_FILE)

if len(RECOLA_invariant_list) > 0:
    print("\n-------- Invariant Features --------")
    for feature in RECOLA_invariant_list:
        print(f" {feature}")

    # STEP 3: MODEL PREP
    PARTICIPANT_GROUPS = list(dataset["Participant_Number"])
    FEATURES = dataset.filter(regex=f'^{"ComPar"}|{"audio_speech"}|{"VIDEO"}|{"Face_detection"}|{"ECG"}|{"EDA"}', axis=1) 
    INVARIANT_FEATURES = dataset.filter(RECOLA_invariant_list, axis=1)           
    if "Class_Label_Valence" in dataset.columns: TARGET_LABEL = dataset["Class_Label_Valence"]
    elif "Class_Label_Arousal" in dataset.columns: TARGET_LABEL = dataset["Class_Label_Arousal"]
    FOLDS = len(set(PARTICIPANT_GROUPS))

    # STEP 4: RUN LOGISITIC REGRESSION EXPERIMENTS
    run_logistic_regression(PARTICIPANT_GROUPS, FEATURES, INVARIANT_FEATURES, TARGET_LABEL, FOLDS)

    # STEP 5: RUN NEURAL NETWORK EXPERIMENTS
    run_neural_networks(PARTICIPANT_GROUPS, FEATURES, INVARIANT_FEATURES, TARGET_LABEL, FOLDS)

else:
    print("\n-------- No Invariant Features were found --------")

# STEP 6: SAVE OUTPUT
sys.stdout = sys.__stdout__
with open(f"Test_Results/{output_file}.txt", "w") as f:
    f.write(buffer.getvalue())

In [9]:
# TEST CASE 4: FEMALE PARTICIPANTS, TARGET LABEL: AROUSAL
output_file = "RECOLA_Test_Case_4"
buffer = io.StringIO()
sys.stdout = buffer

print("-------- TEST CASE 4: FEMALE PARTICIPANTS, TARGET LABEL: AROUSAL --------\n")

# STEP 1: OBTAIN DATASET 
dataset_manager.temproary_recola = dataset_manager.split_by_gender(gender="Female")
dataset_manager.temproary_recola = dataset_manager.remove_class_label(label_to_keep="Arousal")
dataset = dataset_manager.save_custom_dataframe(file_name=f"{output_file}.csv")

# STEP 2: OBTAIN INVARIANT FEATURES
INPUT_FILE = f"Formatted_Datasets/{output_file}.csv"
OUTPUT_FILE = f"Invariant_Features/inv_feat_{output_file}.csv"

RECOLA_invariant_list = get_invariant_features(input_file=INPUT_FILE, output_file=OUTPUT_FILE)

if len(RECOLA_invariant_list) > 0:
    print("\n-------- Invariant Features --------")
    for feature in RECOLA_invariant_list:
        print(f" {feature}")

    # STEP 3: MODEL PREP
    PARTICIPANT_GROUPS = list(dataset["Participant_Number"])
    FEATURES = dataset.filter(regex=f'^{"ComPar"}|{"audio_speech"}|{"VIDEO"}|{"Face_detection"}|{"ECG"}|{"EDA"}', axis=1) 
    INVARIANT_FEATURES = dataset.filter(RECOLA_invariant_list, axis=1)           
    if "Class_Label_Valence" in dataset.columns: TARGET_LABEL = dataset["Class_Label_Valence"]
    elif "Class_Label_Arousal" in dataset.columns: TARGET_LABEL = dataset["Class_Label_Arousal"]
    FOLDS = len(set(PARTICIPANT_GROUPS))

    # STEP 4: RUN LOGISITIC REGRESSION EXPERIMENTS
    run_logistic_regression(PARTICIPANT_GROUPS, FEATURES, INVARIANT_FEATURES, TARGET_LABEL, FOLDS)

    # # STEP 5: RUN NEURAL NETWORK EXPERIMENTS
    run_neural_networks(PARTICIPANT_GROUPS, FEATURES, INVARIANT_FEATURES, TARGET_LABEL, FOLDS)

else:
    print("\n-------- No Invariant Features were found --------")

# STEP 6: SAVE OUTPUT
sys.stdout = sys.__stdout__
with open(f"Test_Results/{output_file}.txt", "w") as f:
    f.write(buffer.getvalue())

In [10]:
# TEST CASE 5: FEMALE PARTICIPANTS, TARGET LABEL: VALENCE
output_file = "RECOLA_Test_Case_5"
buffer = io.StringIO()
sys.stdout = buffer

print("-------- TEST CASE 5: FEMALE PARTICIPANTS, TARGET LABEL: VALENCE --------\n")

# STEP 1: OBTAIN DATASET 
dataset_manager.temproary_recola = dataset_manager.split_by_gender(gender="Female")
dataset_manager.temproary_recola = dataset_manager.remove_class_label(label_to_keep="Valence")
dataset = dataset_manager.save_custom_dataframe(file_name=f"{output_file}.csv")

# STEP 2: OBTAIN INVARIANT FEATURES
INPUT_FILE = f"Formatted_Datasets/{output_file}.csv"
OUTPUT_FILE = f"Invariant_Features/inv_feat_{output_file}.csv"

RECOLA_invariant_list = get_invariant_features(input_file=INPUT_FILE, output_file=OUTPUT_FILE)

if len(RECOLA_invariant_list) > 0:
    print("\n-------- Invariant Features --------")
    for feature in RECOLA_invariant_list:
        print(f" {feature}")

    # STEP 3: MODEL PREP
    PARTICIPANT_GROUPS = list(dataset["Participant_Number"])
    FEATURES = dataset.filter(regex=f'^{"ComPar"}|{"audio_speech"}|{"VIDEO"}|{"Face_detection"}|{"ECG"}|{"EDA"}', axis=1) 
    INVARIANT_FEATURES = dataset.filter(RECOLA_invariant_list, axis=1)           
    if "Class_Label_Valence" in dataset.columns: TARGET_LABEL = dataset["Class_Label_Valence"]
    elif "Class_Label_Arousal" in dataset.columns: TARGET_LABEL = dataset["Class_Label_Arousal"]
    FOLDS = len(set(PARTICIPANT_GROUPS))

    # STEP 4: RUN LOGISITIC REGRESSION EXPERIMENTS
    run_logistic_regression(PARTICIPANT_GROUPS, FEATURES, INVARIANT_FEATURES, TARGET_LABEL, FOLDS)

    # STEP 5: RUN NEURAL NETWORK EXPERIMENTS
    run_neural_networks(PARTICIPANT_GROUPS, FEATURES, INVARIANT_FEATURES, TARGET_LABEL, FOLDS)

else:
    print("\n-------- No Invariant Features were found --------")

# STEP 6: SAVE OUTPUT
sys.stdout = sys.__stdout__
with open(f"Test_Results/{output_file}.txt", "w") as f:
    f.write(buffer.getvalue())

In [11]:
#  TEST CASE 6: MALE AND FEMALE ENVIRONS (2), TARGET LABEL: AROUSAL
output_file = "RECOLA_Test_Case_6"
buffer = io.StringIO()
sys.stdout = buffer

print("-------- TEST CASE 6: MALE AND FEMALE ENVIRONS (2), TARGET LABEL: AROUSAL --------\n")

# STEP 1: OBTAIN DATASET 
male_dataset = dataset_manager.split_by_gender(gender="Male")
female_dataset = dataset_manager.split_by_gender(gender="Female")
dataset_manager.temproary_recola = dataset_manager.simplify_environments(datasets=[male_dataset, female_dataset])
dataset_manager.temproary_recola = dataset_manager.remove_class_label(label_to_keep="Arousal")
dataset = dataset_manager.save_custom_dataframe(file_name=f"{output_file}.csv")

# STEP 2: OBTAIN INVARIANT FEATURES
INPUT_FILE = f"Formatted_Datasets/{output_file}.csv"
OUTPUT_FILE = f"Invariant_Features/inv_feat_{output_file}.csv"

RECOLA_invariant_list = get_invariant_features(input_file=INPUT_FILE, output_file=OUTPUT_FILE)

if len(RECOLA_invariant_list) > 0:
    print("\n-------- Invariant Features --------")
    for feature in RECOLA_invariant_list:
        print(f" {feature}")

if len(RECOLA_invariant_list) > 0:
    print("\n-------- Invariant Features --------")
    for feature in RECOLA_invariant_list:
        print(f" {feature}")

    # STEP 3: MODEL PREP
    PARTICIPANT_GROUPS = list(dataset["Participant_Number"])
    FEATURES = dataset.filter(regex=f'^{"ComPar"}|{"audio_speech"}|{"VIDEO"}|{"Face_detection"}|{"ECG"}|{"EDA"}', axis=1) 
    INVARIANT_FEATURES = dataset.filter(RECOLA_invariant_list, axis=1)           
    if "Class_Label_Valence" in dataset.columns: TARGET_LABEL = dataset["Class_Label_Valence"]
    elif "Class_Label_Arousal" in dataset.columns: TARGET_LABEL = dataset["Class_Label_Arousal"]
    FOLDS = len(set(PARTICIPANT_GROUPS))

    # STEP 4: RUN LOGISITIC REGRESSION EXPERIMENTS
    run_logistic_regression(PARTICIPANT_GROUPS, FEATURES, INVARIANT_FEATURES, TARGET_LABEL, FOLDS)

    # STEP 5: RUN NEURAL NETWORK EXPERIMENTS
    run_neural_networks(PARTICIPANT_GROUPS, FEATURES, INVARIANT_FEATURES, TARGET_LABEL, FOLDS)

else:
    print("\n-------- No Invariant Features were found --------")

# STEP 6: SAVE OUTPUT
sys.stdout = sys.__stdout__
with open(f"Test_Results/{output_file}.txt", "w") as f:
    f.write(buffer.getvalue())

In [ ]:
#  TEST CASE 7: MALE AND FEMALE ENVIRONS (2), TARGET LABEL: VALENCE
output_file = "RECOLA_Test_Case_7"
buffer = io.StringIO()
sys.stdout = buffer

print("-------- TEST CASE 7: MALE AND FEMALE ENVIRONS (2), TARGET LABEL: VALENCE --------\n")

# STEP 1: OBTAIN DATASET 
male_dataset = dataset_manager.split_by_gender(gender="Male")
female_dataset = dataset_manager.split_by_gender(gender="Female")
dataset_manager.temproary_recola = dataset_manager.simplify_environments(datasets=[male_dataset, female_dataset])
dataset_manager.temproary_recola = dataset_manager.remove_class_label(label_to_keep="Valence")
dataset = dataset_manager.save_custom_dataframe(file_name=f"{output_file}.csv")

# STEP 2: OBTAIN INVARIANT FEATURES
INPUT_FILE = f"Formatted_Datasets/{output_file}.csv"
OUTPUT_FILE = f"Invariant_Features/inv_feat_{output_file}.csv"

RECOLA_invariant_list = get_invariant_features(input_file=INPUT_FILE, output_file=OUTPUT_FILE)

if len(RECOLA_invariant_list) > 0:
    print("\n-------- Invariant Features --------")
    for feature in RECOLA_invariant_list:
        print(f" {feature}")

    # STEP 3: MODEL PREP
    PARTICIPANT_GROUPS = list(dataset["Participant_Number"])
    FEATURES = dataset.filter(regex=f'^{"ComPar"}|{"audio_speech"}|{"VIDEO"}|{"Face_detection"}|{"ECG"}|{"EDA"}', axis=1) 
    INVARIANT_FEATURES = dataset.filter(RECOLA_invariant_list, axis=1)           
    if "Class_Label_Valence" in dataset.columns: TARGET_LABEL = dataset["Class_Label_Valence"]
    elif "Class_Label_Arousal" in dataset.columns: TARGET_LABEL = dataset["Class_Label_Arousal"]
    FOLDS = len(set(PARTICIPANT_GROUPS))

    # STEP 4: RUN LOGISITIC REGRESSION EXPERIMENTS
    run_logistic_regression(PARTICIPANT_GROUPS, FEATURES, INVARIANT_FEATURES, TARGET_LABEL, FOLDS)

    # STEP 5: RUN NEURAL NETWORK EXPERIMENTS
    run_neural_networks(PARTICIPANT_GROUPS, FEATURES, INVARIANT_FEATURES, TARGET_LABEL, FOLDS)

else:
    print("\n-------- No Invariant Features were found --------")

# STEP 6: SAVE OUTPUT
sys.stdout = sys.__stdout__
with open(f"Test_Results/{output_file}.txt", "w") as f:
    f.write(buffer.getvalue())